In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import matplotlib.pyplot as plt
import random
import gym
from gym import spaces

from sklearn.preprocessing import LabelEncoder

In [2]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        my_buffer = []
        for i in range(batch_size):
            my_buffer.append(self.buffer[-batch_size+i])
        return my_buffer

    def __len__(self):
        return len(self.buffer)

In [3]:
import csv

menu_dict = {}
first = True
with open("./menu.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if first:
          first=False
          continue
        idx, menu = row
        menu_dict[int(idx)] = menu
menu_dict

{0: '1명 닭꼬치',
 1: '간장치킨',
 2: '갈비탕',
 3: '강된장보리밥',
 4: '고구마치즈돈가스',
 5: '고구마피자',
 6: '고기',
 7: '고등어회',
 8: '곱창',
 9: '국밥',
 10: '김치볶음밥',
 11: '김치전',
 12: '김치찌개',
 13: '낙곱새',
 14: '냉면',
 15: '냉소바',
 16: '닭갈비',
 17: '닭강정',
 18: '닭꼬치',
 19: '닭똥집튀김',
 20: '닭발',
 21: '닭볶음탕',
 22: '닭우동',
 23: '대창',
 24: '덮밥',
 25: '돈가스',
 26: '돼지갈비',
 27: '돼지고기김치찜',
 28: '돼지국밥',
 29: '된장찌개',
 30: '두부김치',
 31: '떡볶이',
 32: '라면',
 33: '로제떡볶이',
 34: '로제마라샹궈',
 35: '마라떡볶이',
 36: '마라탕',
 37: '마제소바',
 38: '막국수',
 39: '멘보샤',
 40: '무뼈닭발',
 41: '바게트',
 42: '바나나 한개',
 43: '밥버거',
 44: '백반',
 45: '봉구스밥버거',
 46: '부대찌개',
 47: '분짜',
 48: '불고기백반',
 49: '불고기피자',
 50: '불닭게티',
 51: '비빔국수',
 52: '사케동',
 53: '삼각김밥',
 54: '삼겹살',
 55: '새우초밥',
 56: '샌드위치',
 57: '선지해장국',
 58: '설렁탕',
 59: '소고기덮밥',
 60: '소고기무국',
 61: '소곱창',
 62: '소바',
 63: '솥밥',
 64: '순대국',
 65: '순대국밥',
 66: '순두부찌개',
 67: '스테이크',
 68: '스파게티',
 69: '쌀국수',
 70: '씨리얼',
 71: '야끼소바',
 72: '양꼬치',
 73: '양념치킨',
 74: '양송이수프',
 75: '연어덮밥',
 76: '연어초밥',
 77: '연어회',
 78: '열무국수',
 7

In [4]:
# DQN 모델 정의 (이전에 정의한 것과 동일해야 함)
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# 에이전트 정의 (이전에 정의한 것과 동일해야 함)
class DQNAgent:
    def __init__(self, state_dim, action_dim, replay_buffer):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.replay_buffer = replay_buffer
        self.model = DQN(state_dim, action_dim)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0005)
        self.epsilon = 0.1  # 사용자 입력에서는 낮은 탐색 확률 사용
        self.epsilon_min = 0.01

    def select_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            q_values = self.model(state)
        action = q_values.max(1)[1].item()
        # top_k = 3
        # _, top_k_indices = q_values.topk(top_k, dim=1, largest=True, sorted=True)

        # # top_k_indices는 (1, 3) 형태의 텐서이므로, 이를 리스트로 변환
        # top_k_actions = top_k_indices.squeeze().tolist()

        return action #q_values #top_k_actions

    def update_model(self, batch_size):
        if len(self.replay_buffer) < batch_size:
            return
        batch = self.replay_buffer.sample(batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(states)
        next_states = torch.FloatTensor(next_states)
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        dones = torch.FloatTensor(dones)
        
        print(self.model(states), actions.view(-1, 1))
        current_q_values = self.model(states).gather(1, actions.view(-1, 1)).squeeze(1)
        max_next_q_values = self.model(next_states).max(1)[0]
        expected_q_values = rewards + (1 - dones) * 0.99 * max_next_q_values

        loss = nn.MSELoss()(current_q_values, expected_q_values.detach())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

# 모델 및 에이전트 초기화
state_dim = 6  # 상태 차원 (예시)
action_dim = len(menu_dict)  # 행동 차원 (예시, 메뉴 수)
replay_buffer = ReplayBuffer(10000)
agent = DQNAgent(state_dim, action_dim, replay_buffer)

# 체크포인트에서 모델 불러오기
checkpoint_path = './model/model_90000.pth'  # 예시 체크포인트 경로
checkpoint = torch.load(checkpoint_path)
agent.model.load_state_dict(checkpoint)
agent.model.eval()

DQN(
  (fc1): Linear(in_features=6, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=116, bias=True)
)

In [9]:
def get_user_input():
    feeling = float(input("기분을 숫자로 입력하세요 (예: 1~10, 1: 최악, 10: 최고): "))
    weather = float(input("날씨를 숫자로 입력하세요 (예: 0: 맑음, 1: 구름, 2: 흐림, 3: 비, 4: 눈): "))
    temp = float(input("체감 온도를 숫자로 입력하세요 (예: 1~10, 1: 추움, 10: 더움): "))
    time = float(input("체감 식사 시간대를 입력하세요 (0: 아침, 1: 점심, 2: 저녁, 3: 야식): "))
    num_people = float(input("인원수를 숫자로 입력하세요 (예: 1, 2, 3, 4 or 5명 이상: 5): "))
    gender = float(input("성별을 입력하세요 (0: 남성, 1: 여성): "))
    state = [feeling, weather, temp, time, num_people, gender]
    return state

def get_user_feedback(action):
    recommended_menus = menu_dict[action]
    feedback = int(input(f"추천된 메뉴: {recommended_menus}. 만족하셨나요? (1: 예, 0: 아니오): "))
    return feedback

In [46]:
# 사용자 입력 받기
state = get_user_input()

# 모델을 통해 메뉴 추천 받기
action = agent.select_action(state)

# 사용자 피드백 받기
reward = get_user_feedback(action)
done = True  # 단일 스텝이므로 에피소드 종료

# 리플레이 버퍼에 경험 추가
next_state = state  # 단일 스텝이므로 상태가 변하지 않음
agent.replay_buffer.push(state, action, reward, next_state, done)

ValueError: could not convert string to float: ''

In [47]:
# 사용자 피드백을 바탕으로 모델 업데이트   아직 구현 안됨
batch_size = 1  # 단일 사용자 피드백이므로 배치 크기를 1로 설정
print(action)
agent.update_model(batch_size)

58
tensor([[-1.0483, -0.6977, -1.1457, -2.8623, -1.1725, -0.8284, -3.2542, -1.5048,
         -0.8813, -1.7878, -1.0156, -1.0621, -1.0196, -0.5850, -3.8578, -0.7260,
         -1.1485, -0.7626, -0.7813, -1.1021,  0.2316, -0.7286, -0.8914, -1.2474,
         -4.2757, -1.2856, -0.7153, -0.6166, -1.2366,  0.3245, -0.6106, -0.8729,
         -1.5315, -0.9760, -0.8006, -0.7005, -6.8486, -1.4681, -1.0068, -0.8387,
         -1.7251, -0.6253, -0.8369, -0.9200, -0.6716, -0.9000, -1.1575, -1.0404,
         -1.0039, -0.8434, -0.3321, -0.8684, -0.7704, -0.5130, -1.3149, -3.1739,
         -1.1124, -0.7156,  0.3601, -1.1145, -0.6231, -0.9737, -0.8301, -0.8180,
         -0.8332, -1.2536, -0.8447, -0.9471, -0.5816, -0.9852, -1.7915, -0.9489,
         -0.8285,  0.1692, -1.0763, -0.7378, -1.1005, -3.5927, -2.6733,  0.1177,
         -1.6377, -1.0329, -1.2822, -1.4865, -0.5618, -1.0235, -2.4817, -0.7085,
         -0.9686, -0.6939, -1.0658, -0.4942, -0.9078, -0.7557, -0.7412, -0.8948,
          0.3411, -0.5615